# Creating a simple forecasting model
Before we can use mosqlient, we need to install the reticulate library in R. If you already have it installed, you can skip the cell below.

In [1]:
install.packages('pak')
library(pak)
pak::pak("rstudio/reticulate")

Installing package into ‘/home/fccoelho/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

! Using bundled GitHub PAT. Please add your own PAT using `gitcreds::gitcreds_set()`.


✔ Updated metadata database: 1.38 MB in 1 file.


ℹ Updating metadata database
✔ Updating metadata database ... done


 

✔ All system requirements are already installed.

  

ℹ No downloads are needed

✔ 1 pkg + 11 deps: kept 10 [4.8s]



In [1]:
library(reticulate)

## Installing Mosqlient

In [2]:
py_install("mosqlient>=1.2.2", envname='r-reticulate')

Using virtual environment 'r-reticulate' ...


+ /home/fccoelho/.virtualenvs/r-reticulate/bin/python -m pip install --upgrade --no-user 'mosqlient>=1.2.2'



In [3]:
mosq <- import("mosqlient")

In [4]:
mosq$version

[1] "1.2.1"

## Downloading the Dengue dataset
Here we will build a simple univariate model using only dengue cases. We could use the Mosqlient library to fetch the data straight from the API, but instead we will download the prepared datasets made available by the sprint organizers. For this you can go to [info.dengue.mat.br/minio](https://info.dengue.mat.br/minio), and download the infodengue dataset. This dataset is a gzip compressed csv file called `infodengue.csv.gv` and save it in the same directory as this notebook.

In [21]:
library(data.table)

In [22]:
data = fread("infodengue.csv.gz")
data

V1,SE,geocode,casos,casprov,Rt,p_rt1,regional,regional_geocode,macroregional,macroregional_geocode,uf
<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
0,201001,4208450,0,NA,0,0,Nordeste,42011,Planalto Norte e Nordeste,4211,SC
1,201001,4216206,0,NA,0,0,Nordeste,42011,Planalto Norte e Nordeste,4211,SC
2,201001,4218509,0,NA,0,0,Meio Oeste,42008,Meio Oeste e Serra Catarinense,4212,SC
3,201001,4216404,0,NA,0,0,Extremo Sul Catarinense,42014,Sul,4210,SC
4,201001,4201307,0,NA,0,0,Nordeste,42011,Planalto Norte e Nordeste,4211,SC
5,201001,4204178,0,NA,0,0,Serra Catarinense,42013,Meio Oeste e Serra Catarinense,4212,SC
6,201001,4219358,0,NA,0,0,Alto Vale do Itajaí,42004,Alto Vale do Itajaí,4216,SC
7,201001,4206405,0,NA,0,0,Extremo Oeste,42001,Grande Oeste,4213,SC
8,201001,4207700,0,NA,0,0,Alto Uruguai Catarinense,42010,Meio Oeste e Serra Catarinense,4212,SC


## Creating a simple Forecast model

In [23]:
library(forecast)
library(tidyverse)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks s